# Style transfer of drawings

- This notebook contains the code to perform style transfer on the drawings selected for training.

In [1]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import numpy as np
from tqdm import tqdm

In [3]:
# utility function to convert an image to a sketch
# from https://towardsdatascience.com/generate-pencil-sketch-from-photo-in-python-7c56802d8acb
def img2sketch(img, k_size):
    grey_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Invert Image
    invert_img=cv2.bitwise_not(grey_img)
    # Blur image
    blur_img=cv2.GaussianBlur(invert_img, (k_size,k_size),0)
    # Invert Blurred Image
    invblur_img=cv2.bitwise_not(blur_img)

    # Sketch Image
    sketch_img=cv2.divide(grey_img,invblur_img, scale=256.0)
    
    return sketch_img

In [2]:
# read the manually annotated pairs
data_folder = "./../../data/"
annotations_by_hand = "./../../data/manual_annotation/annotated_drawing_artwork_pairs_without_style_transfer.csv"
uid2path = "./../../data/text_data/uid2path.pkl"

manually_annotated_df = pd.read_csv(annotations_by_hand)

with open(uid2path, "rb") as outfile:
    uid2path_df = pickle.load(outfile)

For each drawing that is paired with an artwork, it is converted oilpainting, watercolor, textured, pencil sketch and grayscale images and stored in a folder `Train_00` in the same folder that stores the other `Train_XX` folders.

In [4]:
manually_annotated_df_copy = manually_annotated_df.copy()

manually_annotated_df_copy["parent"] = manually_annotated_df_copy["drawing_id"]

modified_images_path = "./../../data/image_data/drawings/TRAIN_00/modified_drawings/"

for rid, row in tqdm(manually_annotated_df.iterrows(), total=len(manually_annotated_df)):
    if row["pair"]:
        drawing_name = row["drawing_id"]
        artwork_name = row["artwork_id"]
        drawing_path = uid2path_df[uid2path_df["uid"]==drawing_name]["path"].iloc[0]
        drawing_full_path = data_folder + drawing_path
        drw_img = cv2.imread(drawing_full_path, cv2.IMREAD_COLOR)

        oil_img = cv2.xphoto.oilPainting(drw_img, 5, 1)
        oilpainted_drawing_name = drawing_name + "_oil"
        cv2.imwrite(modified_images_path + oilpainted_drawing_name + ".jpg", oil_img)
        
        manually_annotated_df_copy.loc[len(manually_annotated_df_copy)] = [oilpainted_drawing_name, artwork_name, True, drawing_name]

        water_img = cv2.stylization(drw_img, sigma_s=60, sigma_r=0.6)
        watercolored_drawing_name = drawing_name + "_water"
        cv2.imwrite(modified_images_path + watercolored_drawing_name + ".jpg", water_img)
        
        manually_annotated_df_copy.loc[len(manually_annotated_df_copy)] = [watercolored_drawing_name, artwork_name, True, drawing_name]

        # pencil_img_gray, pencil_img_color = cv2.pencilSketch(drw_img, sigma_s=25, sigma_r=0.1, shade_factor=0.02)
        # pencilsketch_drawing_color_name = drawing_name + "_pencil_color"
        # pencilsketch_drawing_gray_name = drawing_name + "_pencil_gray"
        # cv2.imwrite(modified_images_path + pencilsketch_drawing_color_name + ".jpg", pencil_img_color)
        # cv2.imwrite(modified_images_path + pencilsketch_drawing_gray_name + ".jpg", pencil_img_gray)
        
#         manually_annotated_df_copy.loc[len(manually_annotated_df_copy)] = [pencilsketch_drawing_color_name, artwork_name, True, drawing_name]
#         manually_annotated_df_copy.loc[len(manually_annotated_df_copy)] = [pencilsketch_drawing_gray_name, artwork_name, True, drawing_name]

        textu_img = cv2.detailEnhance(drw_img, sigma_s=60, sigma_r=0.4)
        textured_drawing_name = drawing_name + "_texture"
        cv2.imwrite(modified_images_path + textured_drawing_name + ".jpg", textu_img)
        
        manually_annotated_df_copy.loc[len(manually_annotated_df_copy)] = [textured_drawing_name, artwork_name, True, drawing_name]
        
        ksize = 101
        pencil_sketch_gray = img2sketch(drw_img, ksize)
        pencilsketch_gray_name = drawing_name + "_pencil_gray"
        cv2.imwrite(modified_images_path + pencilsketch_gray_name + ".jpg", pencil_sketch_gray)
        manually_annotated_df_copy.loc[len(manually_annotated_df_copy)] = [pencilsketch_gray_name, artwork_name, True, drawing_name]
        
        grey_img=cv2.cvtColor(drw_img, cv2.COLOR_BGR2GRAY)
        drawing_gray_name = drawing_name + "_gray"
        cv2.imwrite(modified_images_path + drawing_gray_name + ".jpg", grey_img)
        manually_annotated_df_copy.loc[len(manually_annotated_df_copy)] = [drawing_gray_name, artwork_name, True, drawing_name]

100%|████████████████████████████████████████████████████████████████████████████████| 208/208 [05:22<00:00,  1.55s/it]


In [5]:
manually_annotated_df_copy

,drawing_id,artwork_id,pair,parent
0,2017_14-17_1535_IRA_R_C,Vincent_van_Gogh_368,True,2017_14-17_1535_IRA_R_C
1,2019_18-25_1824_COL_a_R_C_crop,Leonardo_da_Vinci_121,True,2019_18-25_1824_COL_a_R_C_crop
2,1995_14-17_0335_FRA_R_C,Vincent_van_Gogh_750,True,1995_14-17_0335_FRA_R_C
3,1995_14-17_0335_FRA_R_C,Vincent_van_Gogh_91,True,1995_14-17_0335_FRA_R_C
4,2019_18-25_1824_COL_d_R_C_crop,johannes-vermeer_the-girl-with-a-pearl-earring,True,2019_18-25_1824_COL_d_R_C_crop
...,...,...,...,...
858,1998_14-17_0627_CHN_R_C_crop_oil,statute_head,True,1998_14-17_0627_CHN_R_C_crop
859,1998_14-17_0627_CHN_R_C_crop_water,statute_head,True,1998_14-17_0627_CHN_R_C_crop
860,1998_14-17_0627_CHN_R_C_crop_texture,statute_head,True,1998_14-17_0627_CHN_R_C_crop
861,1998_14-17_0627_CHN_R_C_crop_pencil_gray,statute_head,True,1998_14-17_0627_CHN_R_C_crop


The metadata file updated with the style transferred drawings is saved to the disk

In [6]:
manually_annotated_df_copy.to_csv(
    "./../../data/manual_annotation/annotated_drawing_artwork_pairs.csv",
    index=False)